In [1]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.9.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.9.0

In [2]:
!pip install elasticsearch -q
!pip install transformers -q

     |████████████████████████████████| 225kB 2.7MB/s 
     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 3.0MB 13.8MB/s 
     |████████████████████████████████| 890kB 24.5MB/s 
     |████████████████████████████████| 1.1MB 41.8MB/s 


In [3]:
from subprocess import Popen, PIPE, STDOUT
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from transformers import BertTokenizer

import os
import time
import pprint
import numpy as np
import tensorflow_hub as hub

In [4]:
es_server = Popen(
  ['elasticsearch-7.9.0/bin/elasticsearch'], 
  stdout = PIPE, stderr = STDOUT,
  preexec_fn = lambda: os.setuid(1))

In [5]:
!curl -X GET "localhost:9200/"

{
  "name" : "5ab60cc47daf",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "ZARYFEBgQaOfHkSkXJDK0Q",
  "version" : {
    "number" : "7.9.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "a479a2a7fce0389512d6a9361301708b92dff667",
    "build_date" : "2020-08-11T21:36:48.204330Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.0",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [6]:
def gen_data():
  with open('/content/gdrive/My Drive/finch/es/free_chat/data/basic.txt') as f:
    for line in f:
      line = line.rstrip()
      q, a = line.split('<SEP>')
      bert_inp = ['[CLS]'] + list(q) + ['[SEP]']
      bert_inp = tokenizer.convert_tokens_to_ids(bert_inp)
      bert_seg = [0] * len(bert_inp)
      bert_mask = [1] * len(bert_inp)
      res, _ = encoder([np.asarray([bert_inp], dtype=np.int32),
                        np.asarray([bert_mask], dtype=np.int32),
                        np.asarray([bert_seg], dtype=np.int32)])
      yield {
        '_index': 'chatbot',
        'question': q,
        'answer': a,
        'question_embedding': res[0].numpy(),}

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese',
                                           lowercase = True,
                                           add_special_tokens = True)

encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_zh_L-12_H-768_A-12/2",
                         trainable=False)

In [9]:
es = Elasticsearch()
print(es.ping())

es.indices.create(index='chatbot')

mapping = {
  'properties': {
    'question': {
      'type': 'text',
    },
    'question_embedding': {
      'type': 'dense_vector',
      'dims': 768,
    },
  }
}
es.indices.put_mapping(body=mapping, index='chatbot')

helpers.bulk(es, gen_data())

True


(366, [])

In [ ]:
while True:
  text_inp = input('Input:')
  t0 = time.time()
  bert_inp = ['[CLS]'] + list(text_inp) + ['[SEP]']
  bert_inp = tokenizer.convert_tokens_to_ids(bert_inp)
  bert_seg = [0] * len(bert_inp)
  bert_mask = [1] * len(bert_inp)
  res, _ = encoder([np.asarray([bert_inp], dtype=np.int32),
                    np.asarray([bert_mask], dtype=np.int32),
                    np.asarray([bert_seg], dtype=np.int32)])
  query_vector = res[0].numpy()
  script_query = {
  'script_score': {
    'query': {'match_all': {}},
    'script': {
      'source': "cosineSimilarity(params.query_vector, doc['question_embedding']) + 1.0",
      'params': {'query_vector': query_vector},
      }
    }
  }
  dsl = {
    'query': script_query,
    '_source': {'excludes': ['question_embedding']},
  }
  hits = es.search(index='chatbot', body=dsl)['hits']['hits']
  print('Match:', hits[0]['_source'])
  print('%.2f sec' % (time.time() - t0))
  print()

Input:爱你


/usr/local/lib/python3.6/dist-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: The vector functions of the form function(query, doc['field']) are deprecated, and the form function(query, 'field') should be used instead. For example, cosineSimilarity(query, doc['field']) is replaced by cosineSimilarity(query, 'field').
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


Match: {'question': '爱你', 'answer': '爱你'}
0.85 sec

Input:我喜欢你
Match: {'question': '我爱你', 'answer': '我也爱你'}
0.15 sec

Input:我爱你
Match: {'question': '我爱你', 'answer': '我也爱你'}
0.13 sec

Input:天气
Match: {'question': '天气', 'answer': '天气还行 要查具体的吗'}
0.12 sec

Input:几点了
Match: {'question': '现在几点了', 'answer': '要查一下现在的具体时间吗'}
0.12 sec

Input:几岁了
Match: {'question': '你几岁了', 'answer': '你猜呢'}
0.13 sec

Input:笑话
Match: {'question': '笑话', 'answer': '白天文明 但不精神 晚上精神 但不文明'}
0.13 sec

Input:天气
Match: {'question': '天气', 'answer': '天气还行 要查具体的吗'}
0.12 sec

Input:天气好
Match: {'question': '天气真好', 'answer': '适合出门走动走动'}
0.12 sec

Input:天气糟糕
Match: {'question': '天气太糟糕了', 'answer': '最好不要出门'}
0.12 sec

Input:冷
Match: {'question': '冷', 'answer': '多穿衣服哈'}
0.11 sec

Input:热
Match: {'question': '热', 'answer': '对啊, 热死人了'}
0.11 sec

Input:热死了
Match: {'question': '笑死我了', 'answer': '瞧把你开心的'}
0.11 sec

Input:激动死了
Match: {'question': '太激动了', 'answer': '发生什么好事了 说来听听'}
0.12 sec

Input:我伤心了
Match: {'question': '好难受', 'answer': 